# Lab 5 - metryki w przestrzeni napisów

Zadanie dotyczy różnych metryk w przestrzeni napisów.

1. Zaimplementuj przynajmniej 3 metryki spośród wymienionych: cosinusowa, LCS, DICE, euklidesowa.
2. Zaimplementuj przynajmniej 2 sposoby oceny jakości klasteryzacji (np. indeks Daviesa-Bouldina).
3. Stwórz stoplistę najczęściej występujących słów.
4. Wykonaj klasteryzację zawartości załączonego pliku (lines.txt) przy użyciu przynajmniej 2 algorytmów oraz metryk zaimplementowanych w pkt. 1. i metryki Levenshteina. Każda linia to adres pocztowy firmy, różne sposoby zapisu tego samego adresu powinny się znaleźć w jednym klastrze.
5. Porównaj jakość wyników sposobami zaimplementowanymi w pkt. 2.
6. Czy masz jakiś pomysł na poprawę jakości klasteryzacji w tym zadaniu?

Sprawozdanie powinno zawierać porównanie wyników wszystkich metryk z użyciem stoplisty i bez.
Można jako wzorcową klasteryzację użyć pliku clusters.txt.

In [1]:
import numpy as np
from sklearn.cluster import DBSCAN

### N-gramy

In [2]:
def n_gram_dict(text, n, d = {}):
    l = len(text)

    for j in range(l-n+1):
        t = text[j:j+n]
        if t in d.keys():
            d[t] += 1
        else:
            d[t] = 1

    return d

In [3]:
n_gram_dict("texttext", 3)

{'tex': 2, 'ext': 2, 'xtt': 1, 'tte': 1}

## Metryki

### Metryka LCS

In [4]:
def LCS(text1, text2):
    n = len(text1)
    m = len(text2)
    
    dp = [[0]*(m+1) for _ in range(n+1)]
    result = 0
    for i in range(1, n+1):
        for j in range(1, m+1):
            if text1[i-1] == text2[j-1]:
                dp[i][j] = dp[i-1][j-1] + 1
                result = max(result, dp[i][j])
            else:
                dp[i][j] = 0

    return result

In [5]:
def LCS_metric(text1, text2):
    if len(text1)==0 and len(text2)==0:
        return 0
    return 1 - LCS(text1, text2)/max(len(text1), len(text2))

In [6]:
def LCS_matrix(text):
    n = len(text)
    matrix = np.zeros((n,n))
    for i in range(n):
        for j in range(i+1, n):
            matrix[i][j] = matrix[j][i] = LCS_metric(text[i], text[j])
    
    return matrix

In [7]:
LCS_metric("texttextttttttttttttt", "errrrrrrtexttextaaaaaa")

0.6363636363636364

### Matryka cosinusowa

In [8]:
def cosine_metric(vec1, vec2):
    s = 0
    t1_s = 0
    t2_s = 0
    for i in range(len(vec1)):
        s += vec1[i]*vec2[i]
        t1_s += vec1[i]**2
        t2_s += vec2[i]**2
    if t1_s != 0 and t2_s != 0:
        return 1 - s/(np.sqrt(t1_s) * np.sqrt(t2_s))
    else:
        return 1
    

In [9]:
def cosine_metric_2(text1, text2, n=3):
    t1_dict = n_gram_dict(text1, n, {})
    t2_dict = n_gram_dict(text2, n, {})
    
    s = 0
    t1_s = 0
    t2_s = 0
    for key in t1_dict.keys():
        if key in t2_dict.keys():
            s += t1_dict[key] * t2_dict[key]
    
    for key in t1_dict.keys():
        t1_s += t1_dict[key]**2
        
    for key in t2_dict.keys():
        t2_s += t2_dict[key]**2
    
    return 1 - s/(np.sqrt(t1_s) * np.sqrt(t2_s))        
    

In [10]:
cosine_metric_2("texttextttttttttttttt", "errrrrrrtexttextaaaaaa", 3)

0.8826862694567411

### Metryka euklidesowa

In [11]:
def euclidean_metric(vec1, vec2):
    s = 0
    for i in range(len(vec1)):
        s += (vec1[i] - vec2[i])**2
    
    return np.sqrt(s)
            
    
    

In [12]:
def euclidean_metric_2(text1, text2, n=3):
    t1_dict = n_gram_dict(text1, n)
    t2_dict = n_gram_dict(text2, n)
    
    s = 0
    for key in t1_dict.keys():
        if key in t2_dict.keys():
            s += (t1_dict[key] - t2_dict[key])**2
        else:
            s += t1_dict[key]**2
            
    for key in t2_dict.keys():
        if key not in t1_dict.keys():
            s += t2_dict[key]**2
    return np.sqrt(s)
            
    
    

### Metryka Dice

In [14]:
def dice_metric(vec1, vec2):
    s = 0
    for i in range(len(vec1)):
        s += min(vec1[i], vec2[i])
        
    if np.sum(vec1) + np.sum(vec2) == 0.0:
        return 0
    return 1 - 2*s/(np.sum(vec1)+np.sum(vec2))

### Odległość Levenshteina

In [15]:
def Levenshtein_distance(text1, text2):
    n = len(text1)
    m = len(text2)  
    dp = [[0]*(n+1) for _ in range(m+1)]
    for i in range(n+1):
        dp[0][i]=i
    for i in range(m+1):
        dp[i][0] = i
    
    for i in range(1,m+1):
        for j in range(1,n+1):
            if text1[j-1] != text2[i-1]:
                insert = dp[i][j-1] + 1
                remove = dp[i-1][j] + 1
                replace = dp[i-1][j-1] + 1
                dp[i][j] = min(insert, remove, replace)
            else:
                dp[i][j] = dp[i-1][j-1]
    
    return dp[m][n]

In [16]:
def Levenshtein_matrix_dist(text):
    n = len(text)
    matrix = np.zeros((n,n))
    for i in range(n):
        for j in range(i+1, n):
            matrix[i][j] = matrix[j][i] = Levenshtein_distance(text[i], text[j])
    
    return matrix

## Ocena jakości klasteryzacji

In [17]:
# zwraca średnią odległość pomiędzy losowymi elementami z obu klastrów "i" oraz "j"
def clusters_dist(clusters, i, j, metric_function = euclidean_metric):
    d = metric_function(clusters[i][0], clusters[j][0])
    return d

In [18]:
def element_dist(cluster, metric_function = euclidean_metric):
    dist = 0
    i = 0
    n = len(cluster)
    for j in range(n):
        for k in range(j+1, n):
            dist += metric_function(cluster[j], cluster[k])
            i +=1
    if i==0:
        return 0 # tylko 1 element w klastrze
    return dist/i

### Indeks Daviesa-Bouldina

In [19]:
def DBI(clusters, metric_function = euclidean_metric):
    n = len(clusters) #liczba klastrów
    average_element_dist = [0]*n
    for i in range(n):
        average_element_dist[i] = element_dist(clusters[i])
    s = 0
    for i in range(n):
        mx = 0
        for j in range(i+1, n):
            fact = (average_element_dist[i]+average_element_dist[j])/clusters_dist(clusters, i, j)
            mx = max(mx, fact)
        s += mx
    return s/n

### indeks Dunna

In [20]:
def DI(clusters):
    n = len(clusters)
    mn = 1000000 # min odległość pomiędzy klastrami
    for i in range(n):
        for j in range(i+1, n):
            mn = min(mn, clusters_dist(clusters, i, j))
            
    mx = 0   # max odległość między elementami w klastrze -> przyjmuję tu max średnią odległość 
             # pomiędzy elementami w klastrze
        
    average_element_dist = [0]*n
    for i in range(n):
        average_element_dist[i] = element_dist(clusters[i])
    mx = max(average_element_dist)
    
    return mn/mx

## Stoplista

In [21]:
def get_sorted_stoplist():
    file = open('lines_short.txt')
    text1 = file.read()
    text1 = text1.split(" ")
    file.close()

    d = {}
    for word in text1:
        if word in d.keys():
            d[word]+=1
        else:
            d[word] = 1
        
    return dict(sorted(d.items(), key=lambda item: -item[1]))

In [22]:
def delete_words(n): # n - liczba słów do usunięcia
    
    d = get_sorted_stoplist()
    to_delete = set()
    j = 0
    for i in d.keys():
        to_delete.add(i)
        j +=1
        if j > n:
            break
    
    file = open('lines_short.txt')
    text1 = file.read()
    text1 = text1.split("\n")
    file.close()
    
    lines_n = len(text1)
    new_text = [None]*lines_n
    i = 0
    for line in text1:
        lin = line.split(" ")
        for j in range(len(lin)):
            word = lin[j]
            if word in to_delete:
                lin[j] = ''
        lin = ''.join(lin)
        new_text[i] = lin
        i +=1
        
    return new_text
        

## Klasteryzacja

In [23]:
file = open('lines_short.txt')
text = file.read()
text = text.split('\n')
file.close()

In [24]:
text_s = delete_words(10)

In [25]:
def prepare_X(text):
    d = {}
    for line in text:
        d = n_gram_dict(line, 4, d)

    i = 0    
    for key in d.keys():
        d[key] = i
        i +=1
    n = len(text)-1
    keys_num = len(d.keys())
    
    X = np.zeros((n, keys_num))
    i = 0

    for line in text:
        if len(line)<=2:
            break
        line_d = n_gram_dict(line, 4, {})
    
        for key in line_d.keys():
            X[i][d[key]] = line_d[key]
        i +=1
        
    return X

In [26]:
def convert_labels(X, labels):
    i = 1
    while labels[-i] == -1:
        i+=1
    n = labels[-i]
    clusters = [None]*(n+1)
    for i in range(len(clusters)):
        clusters[i] = []
    for i in range(len(labels)):
        clusters[labels[i]].append(X[i])
    return clusters

In [27]:
X = prepare_X(text)
X_s = prepare_X(text_s)

### Klasteryzacja z metryką euklidesową
    

#### Bez stoplisty

In [28]:
clustering = DBSCAN(eps=7, min_samples=1, metric = euclidean_metric).fit(X)
labels = clustering.labels_
print(labels)
clusters = convert_labels(X,labels)
print(DBI(clusters))
print(DI(clusters))

[ 0  1  2  2  3  4  5  6  7  8  9 10 11 11 11 11 11 11 11 12 12 13 14 15
 16 17 18 19 19 20 21 22 23 24]
0.3304436663402599
1.1981825350018183


#### Po usunięciu wyrazów ze stoplisty

In [29]:
clustering = DBSCAN(eps=7, min_samples=1, metric = euclidean_metric).fit(X_s)
labels = clustering.labels_
print(labels)
clusters = convert_labels(X_s,labels)
print(DBI(clusters))
print(DI(clusters))

[ 0  1  2  2  3  4  5  6  7  8  9 10 11 11 11 11 11 11 11 12 12 13 14 15
 16 17 18 19 19 20 21 22 23 24]
0.23383988909126718
1.2360330811826106


### Klasteryzacja z metryką cosinusową

#### Bez stoplisty

In [30]:
clustering = DBSCAN(eps=0.3, min_samples=1, metric = cosine_metric).fit(X)
labels = clustering.labels_
print(labels)
clusters = convert_labels(X,labels)
print(DBI(clusters))
print(DI(clusters))

[ 0  1  2  2  3  4  5  6  7  8  9 10 11 11 11 11 11 11 11 12 12 13 14 15
 16 17 18 19 19 20 21 22 23 -1]
0.7884933101294617
0.9830496196132974


#### Po usunięciu wyrazów ze stoplisty

In [31]:
clustering = DBSCAN(eps=0.3, min_samples=1, metric = cosine_metric).fit(X_s)
labels = clustering.labels_
print(labels)
clusters = convert_labels(X_s,labels)
print(DBI(clusters))
print(DI(clusters))

[ 0  1  2  2  3  4  5  6  7  8  9 10 11 11 11 11 11 11 11 12 12 13 14 15
 16 17 18 19 19 20 21 22 23 -1]
0.7404031264825819
1.0488088481701516


### Klasteryzacja z metryką Dice

#### Bez stoplisty

In [32]:
clustering = DBSCAN(eps=0.3, min_samples=1, metric = dice_metric).fit(X)
labels = clustering.labels_
print(labels)
clusters = convert_labels(X,labels)
print(DBI(clusters))
print(DI(clusters))

[ 0  1  2  2  3  4  5  6  7  8  9 10 11 11 11 11 11 11 11 12 12 13 14 15
 16 17 18 19 19 20 21 22 23 24]
0.3304436663402599
1.1981825350018183


#### Po usunięciu wyrazów ze stoplisty

In [33]:
clustering = DBSCAN(eps=0.3, min_samples=1, metric = dice_metric).fit(X_s)
labels = clustering.labels_
print(labels)
clusters = convert_labels(X_s,labels)
print(DBI(clusters))
print(DI(clusters))

[ 0  1  2  2  3  4  5  6  7  8  9 10 11 11 11 11 11 11 11 12 12 13 14 15
 16 17 18 19 19 20 21 22 23 24]
0.23383988909126718
1.2360330811826106


### Klasteryzacja z odległością Levensteina

#### Bez stoplisty

In [34]:
matrix = Levenshtein_matrix_dist(text)
clustering = DBSCAN(eps=7, min_samples=1).fit(X, matrix)
labels = clustering.labels_
print(labels)
clusters = convert_labels(X,labels)
print(DBI(clusters))
print(DI(clusters))

[ 0  1  2  2  3  4  5  6  7  8  9 10 11 11 11 11 11 11 11 12 12 13 14 15
 16 17 18 19 19 20 21 22 23 24]
0.3304436663402599
1.1981825350018183


#### Po usunięciu wyrazów ze stoplisty

In [35]:
matrix = Levenshtein_matrix_dist(text_s)
clustering = DBSCAN(eps=7, min_samples=1).fit(X_s, matrix)
labels = clustering.labels_
print(labels)
clusters = convert_labels(X_s,labels)
print(DBI(clusters))
print(DI(clusters))

[ 0  1  2  2  3  4  5  6  7  8  9 10 11 11 11 11 11 11 11 12 12 13 14 15
 16 17 18 19 19 20 21 22 23 24]
0.23383988909126718
1.2360330811826106


### Klasteryzacja z metryką LCS

#### Bez stoplisty

In [36]:
matrix = LCS_matrix(text)
clustering = DBSCAN(eps=7, min_samples=1).fit(X, matrix)
labels = clustering.labels_
print(labels)
clusters = convert_labels(X,labels)
print(DBI(clusters))
print(DI(clusters))

[ 0  1  2  2  3  4  5  6  7  8  9 10 11 11 11 11 11 11 11 12 12 13 14 15
 16 17 18 19 19 20 21 22 23 24]
0.3304436663402599
1.1981825350018183


#### Po usunięciu wyrazów ze stoplisty

In [37]:
matrix = LCS_matrix(text_s)
clustering = DBSCAN(eps=7, min_samples=1).fit(X_s, matrix)
labels = clustering.labels_
print(labels)
clusters = convert_labels(X_s,labels)
print(DBI(clusters))
print(DI(clusters))

[ 0  1  2  2  3  4  5  6  7  8  9 10 11 11 11 11 11 11 11 12 12 13 14 15
 16 17 18 19 19 20 21 22 23 24]
0.23383988909126718
1.2360330811826106


## Podsumowanie

Obliczenia zostały wykonane dla bardzo skróconego pliku, ponieważ zajmowały bardzo dużo czasu, a klasteryzację należało wykonać 10 razy. Z tego względu usunięto tylko 10 pierwszych słów ze stoplisty - najczęściej występujących słów w tekście. Dla tak małej liczby linijek w prawie każdym przypadku klasteryzacja daje poprawny wynik, zostało to osiągnięte dzięki kilkukrotnemu sprawdzaniu najlepszej wartości hiperparametru epsilon dla każdej metryki. Po usunięciu wyrazów ze stoplisty wartość współczynnika DB znacznie się zmniejszyła (o ok. 50%), nastomiast współczynnik Dunna się nieznacznie zwiększył.
Aby poprawić jakość klasteryzacji należałoby wykonać kolejne obliczenia zmieniając np. rozmiar n-gramów (w aktualnej wersji rozważam 4-gramy) sprawdzając jaki rozmiar daje najlepsze rozwiązanie.